# Tarea 8

_Tarea 8_ de _Benjamín Rivera_ para el curso de __Métodos Numéricos__ impartido por _Joaquín Peña Acevedo_. Fecha limite de entrega __1 de Noviembre de 2020__.

In [1]:

import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import seaborn as sns
import scipy as sp
from scipy.linalg import solve_triangular # Para backward y forward substitution

NOTEBOOK = True

## Ejercicio 1

Programar y probar el método de descomposición en valores singulares. Anque algoritmo permite factorizar una matriz rectangular, el programa se va a probar con matrices cuadradas para revisar su número de condición y el uso de la factorización para resolver sistemas de ecuaciones.

### Implementar Algoritmo 1

Escriba una función que implemente el Algoritmo 1 para obtener las matrices U, V y el arreglo s de la descomposición en valores singulares de una matriz A.

In [2]:
def reordenar_matriz(mat, orden):
    if type(mat) == np.matrix:
        return np.concatenate([mat[:,i] for i in orden], 
                              axis=1)
    else:
        return [mat[i] for i in orden]
        

def ordenar(M1: np.matrix, M2: np.matrix, v: list) -> tuple:
    """ Funcion para reordenar las columnas de las
    matrices como se pide en Algoritmo 1.
    """
    # Buscamos el orden en funcion de s O(nlog n)
    orden = [i for i in range(len(v))]
    orden.sort(key=lambda i: v[i],
               reverse=True)
    # -------------------------------
    # Reordenamos los recibidos O(3n)
    M1 = reordenar_matriz(M1, orden)
    M2 = reordenar_matriz(M2, orden)
    v  = reordenar_matriz(v, orden) 
    # --------------------------
    # Regresamos los reordenados
    return M1, M2, v


if NOTEBOOK and True:
    """ Parte para probar el uso de ordenar. """
    n = 3
    U = np.matrix(np.random.rand(n, n), dtype=np.float16)
    V = np.matrix(np.random.rand(n, n), dtype=np.float16)
    s = np.random.rand(1, n).flatten()
    print(U, end='\n\n')
    print(V, end='\n\n')
    print(s, end='\n\n')

    print('-'*n*12)
    U, V, s = ordenar(U, V, s)
    print(U, end='\n\n')
    print(V, end='\n\n')
    print(s, end='\n\n')

[[0.538  0.771 ]
 [0.4976 0.785 ]]

[[0.9023 0.3386]
 [0.744  0.1248]]

[0.70038923 0.98527191]

------------------------
[[0.771  0.538 ]
 [0.785  0.4976]]

[[0.3386 0.9023]
 [0.1248 0.744 ]]

[0.9852719110897654, 0.7003892272142301]



In [3]:
# Algoritmo 1
def Algoritmo1(A: np.matrix, m: int, n: int, T=None, N=100, dtype=np.float64):
    """ Descomopsocion en valores singulares. 
    
    Input:
        A := Matriz de mxn con columnas $a_i, i\in [0,n]$
    """
    # En caso de no pasar tolerancia lo ponemos al epsilon del tipo de dato
    if T is None:
        T = (np.finfo(dtype).eps)**(1/2)
    if not isinstance(A, np.matrix):
        try:
            A = np.matrix(A)
        except:
            raise Exception("=( nshe k pz")

    # Inicializar valores
    V = np.matrix(np.identity(n), 
                  dtype=dtype)
    k = 0
    F = 1

    while k < N and F > 0:
        k = k+1
        F = 0
        for i in range(n-1):
            for j in range(i+1, n):
                alpha = A[:, i].transpose()*A[:, i]
                gamma = A[:, j].transpose()*A[:, j]
                beta  = A[:, i].transpose()*A[:, j]
                
                if alpha*gamma > np.finfo(dtype).eps and abs(beta) > T*alpha*gamma:
                    F = 1
                    if beta != 0:
                        nu = (gamma - alpha)/(2*beta)
                        t = 1/(abs(nu) + np.sqrt(1 + nu**2))
                        if nu < 0:
                            t *= -1
                        c = 1/np.sqrt(1 + t**2)
                        s = t*c
                    else:
                        c = 1; s = 0; t = 0
                    
                    # Modificacion de A
                    a = A[:, i]
                    b = A[:, j]
                    
                    A[:, i] = a*c - b*s
                    A[:, j] = a*s + b*c
                    
                    # Modificacion de V
                    a = V[:, i]
                    b = V[:, j]
                    
                    V[:, i] = a*c - b*s
                    V[:, j] = a*s + b*c
    
    s = []
    for j in range(n):
        s.append( LA.norm(A[:, j], 2) )
        
    A, V, s = ordenar(A, V, s)
    
    U = np.identity(n)
    for j in range(n):
        U[:, j] = np.reshape(A[:, j]/s[j], 
                             (5))

    return U, V, s


### Funcion de solucion

Escriba una función que calcule una aproximación de la solución del sistema $Ax = b$ usando la descomposición en valores singulares de la siguiente manera. La función recibe las matrices $U$ y $V$ , el arreglo $s$ con los valores singulares (calculados con la funcion del inciso anterior), su tamaño n (por matrices cuadradas) y un ı́ndice k. La función debe devolver el vector
$$
    x = \sum_{i=1}^k \frac{u_i^T b}{s_i} v_i
$$
donde $u_i, v_i$ son los i-esimas columnas de las matrices $U, V$ (correpondientemente).

#### Nota

Dado que la expresion requiere del vector $b$, que es el vector de terminos dependientes del sistema, este tambien sera pasado a la funcion a pesar de que no se pido en el enunciado. Hay que tener cuidado con sus dimensiones.

In [4]:
def aproximacion_solucion(U: np.matrix, V: np.matrix, s: list, 
                          b: np.matrix, n: int, k: int):
    return sum([ (U[:,i].transpose()*b)/s[i]*V[:, i] 
               for i in range(k)])

### Interfaz 1

Escriba un programa que reciba desde la lı́nea de comandos el nombre de un archivo que contiene una matriz.

Lea el archivo para crear la matriz $A$ y use la función del primer inciso para calcular su descomposición en valores singulares tomando $\tau = \sqrt{\epsilon_m}$

Imprima la siguiente información:
 1. Dimensiones $m, n$
 2. El valor del error de la ortogonalidad de $U$, $||I - U^TU||$
 3. El valor del error de la ortogonalidad de $V$, $||I - V^TV||$
 4. Crear la matriz $S$ con $s$ en su diagonal e imprimir $||A - USV^T ||$.
 5. El numero de condicion de la matriz $k_2 = \frac{s_1}{s_n}$
 
Puede eligir la norma matricial para calcular los errores.

In [5]:
def load_matrix(file_name: str, 
                path = "datos/", ext=".npy", 
                dtype=np.float64) -> np.matrix:
    """ Funcion para cargar una matriz de un archivo. """
    if ext == ".npy":
        return np.matrix(np.load(path+file_name+ext, 
                                 allow_pickle=True),
                         dtype=dtype)
    
    else:
        """ Sin fomato especifico, esperamoe esta en texto
        con condificacion estandar 'utf-8' e iran siendo 
        ingresados como fuera de esperarse.
        """
        return np.matrix(np.loadtxt(path+file_name+ext),
                         dtype=dtype)

In [6]:
def proceso( file_name: str, path = "datos/", ext=".npy"):
    lon = 100
    dtype = np.float64
    
    # Cargar matriz
    A = load_matrix(file_name, path, ext, 
                    dtype=dtype)
    szA = A.shape
    # Descomposicion
    U, V, s = Algoritmo1(A, szA[0], szA[1])
    
    I = np.identity(szA[1])
    # Errores ortogonalidad
    errU = LA.norm( I - U.transpose()*U)
    errV = LA.norm( I - V.transpose()*V)
    
    # Error de la factorizacion
    S = np.diag(s)
    errS = LA.norm( A - U*S*V.transpose())
    
    # Condicion de matrix
    k2 = s[1]/s[-1]
    
    print('-'*lon)
    print(f"Dimensiones\n\t m={szA[0]}, n={szA[1]}")
    print(f"Errores ortogonaldad")
    print(f"\t U -> {errU}")
    print(f"\t V -> {errV}")
    print(f"Error factorizacion")
    print(f"\t s -> {s}")
    print(f"\t S -> {errS}")
    print(f"Condicion de matriz\n\t {k2}")
    print('-'*lon)
    
    return U, V, s


In [7]:
if True:
    _ = proceso('matA5')
    
    print()
    print(_[0])
    print(_[1])
    print(_[2])

----------------------------------------------------------------------------------------------------
Dimensiones
	 m=5, n=5
Errores ortogonaldad
	 U -> 2.159797139132583
	 V -> 1.2914954894440789
Error factorizacion
	 s -> [5.668182669132584, 5.539783461443808, 3.580430538421286, 0.9564239531731384, 0.45613123053785987]
	 S -> 8.639275173885142
Condicion de matriz
	 12.14515273359252
----------------------------------------------------------------------------------------------------

[[ 0.10398117  0.31954811 -0.25864805  0.87770245  0.22319612]
 [ 0.22477076  0.0194396  -0.88716441 -0.20741222 -0.34499231]
 [-0.8542392   0.44376759 -0.20876335 -0.14546838  0.09274987]
 [-0.32990332 -0.80044237 -0.28505351  0.14900366  0.38340348]
 [-0.31639408 -0.24469052  0.14561203  0.378491   -0.82192642]]
[[ 0.2051436  -0.33894322 -0.20452588  0.609412    0.2277852 ]
 [ 0.10715603 -0.15408889  0.4647943  -0.35385843  0.33778523]
 [-0.01099174  0.3471795  -0.19830619  0.2933302   0.26409681]
 [ 0.7

In [8]:
from scipy.linalg import svdvals

if True:
    A = load_matrix('matA5', dtype=np.float64)
    _ = np.linalg.svd(A)
    
    print()
    print(_[0])
    print(_[1])
    print(_[2])
    
    print(svdvals(A))


[[-0.26838471 -0.27706406  0.05167968 -0.62186223  0.67957468]
 [ 0.04712463 -0.69107825  0.59874707 -0.09076397 -0.39173201]
 [-0.69562312  0.42522753  0.56039866  0.14533697 -0.01097891]
 [ 0.64516575  0.2878811   0.56644176  0.09281572  0.41402247]
 [ 0.16005797  0.42655877  0.06267565 -0.75849303 -0.4617241 ]]
[10.0453801   7.58073142  6.06264573  1.05694463  0.8686526 ]
[[ 0.64592571  0.1089041  -0.71639276 -0.02889685 -0.23846647]
 [-0.23331282  0.25227594 -0.3553973  -0.60494665  0.62422295]
 [ 0.10007065 -0.45205192  0.18905183 -0.76223517 -0.41096457]
 [-0.25781745  0.7743371   0.07353165 -0.18175952 -0.5435888 ]
 [ 0.67220616  0.34720003  0.56508997 -0.13843967  0.29849421]]
[10.0453801   7.58073142  6.06264573  1.05694463  0.8686526 ]
